In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
import plotly.express as px

In [ ]:
### Variables ###

In [ ]:
days = range(5) ###Eventually model will be extended to a month or year
#sessions = range(17)
sessions = range(19)
tracers = range(4)
scanners = range(2)

pib_mk_weight=3
mk_pib_weight=3
mk_concur_weight=3
pib_concur_weight=3

###TODO: Add variable for staff schedules; Design matrix?
#Human Readable
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
#sess_names = ["08:00 - 10:00","08:30 - 10:30","09:00 - 11:00",
#             "09:30 - 11:30","10:00 - 12:00","10:30 - 12:30",
#             "11:00 - 13:00","11:30 - 13:30","12:00 - 14:00",
#             "12:30 - 14:30","13:00 - 15:00","13:30 - 15:30",
#             "14:00 - 16:00","14:30 - 16:30","15:00 - 17:00",
#             "15:30 - 17:30","16:00 - 18:00"]
#Added 2 sessions
sess_names = ["08:00 - 08:30","08:30 - 09:00","09:00 - 09:30",
             "09:30 - 10:00","10:00 - 10:30","10:30 - 11:00",
             "11:00 - 11:30","11:30 - 12:00","12:00 - 12:30",
             "12:30 - 13:00","13:00 - 13:30","13:30 - 14:00",
             "14:00 - 14:30","14:30 - 15:00","15:00 - 15:30",
             "15:30 - 16:00","16:00 - 16:30","16:30 - 17:00",
              "17:00 - 17:30"]
tracer_names = ["PIB","AV1451","MK6240","UCB-J"]
scan_durations = {"PIB":3,"AV1451":3,"MK6240":2,"UCB-J":3}
tracer_isotopes = {"PIB":"11C","AV1451":"18F","MK6240":"18F","UCB-J":"11C"}
scanner_names = ["ECAT HR+","Biograph Horizon"]

In [ ]:
###TODOs###
#Adjust for 30 min sessions - adjust constraints so tracers take up appropriate number of sessions
#Adjust rules discussed with Finn
#Give preference to 13:30 UCB-J over all others
#Use desing matrix to allow uses to block off sessions

In [ ]:
### No use for design matrix yet but could be used to 
#weight individual sessions

design_matrix = []
#sessions
sessions_design = [1]*len(sessions)
#days
days_design = []
for day in range(len(days)):
    days_design.append(sessions_design)
#tracers
tracers_design = []
for tracer in range(len(tracers)):
    tracers_design.append(days_design)
#scanners
for scanner in range(len(scanners)):
    design_matrix.append(tracers_design)
#print(design_matrix)
#design_matrix[p][t][d][s]

In [ ]:
### Creates the model CP Model: 
#https://developers.google.com/optimization/reference/python/sat/python/cp_model
model = cp_model.CpModel()

In [ ]:
### Decision Variables ###

In [ ]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is a Bool of whether day/session/tracer is
#instance is scheduled or not
x = {}
for p in scanners:
    for t in tracers:
        for d in days:
            for s in sessions:
                x[(t,d,s,p)] = model.NewBoolVar(
                    "{}{}{}{}".format(tracer_names[t],
                                      day_names[d],
                                      sess_names[s],
                                      scanner_names[p]))

In [ ]:
#Decision Variables to link outcomes/concurrent end states

In [ ]:
#Allows us to sum instances when a PIB scan is followed by an MK6240 scan in the same day
#Consider removing t1 and t2 since do not change and not necessary; more readable though?
pib = tracer_names.index("PIB")
mk = tracer_names.index("MK6240")
pib_dur=scan_durations["PIB"]
mk_dur=scan_durations["MK6240"]
pib_mk_sess_by_day = {}
for p in scanners:
    for s in sessions:
        for t in [pib,mk]:
            for d  in days:
                pib_mk_sess_by_day[(t,s,d,p)] = model.NewBoolVar("{}{}{}{}".format(tracer_names[t],
                                                                                   day_names[d],
                                                                                   sess_names[s],
                                                                                   scanner_names[p]))
                
###Could remove extra variables for MK sessions before 12:30 or 11:30 Mondays
###Add variables for PIB Followed by MK
pib_mk_concur={}             
for p in scanners:
    for s1 in range(len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan
        for s2 in range(s1+pib_dur,len(sessions)): #MK6240 scan must start after end of PIB
            pib_mk_concur[(pib,s1,mk,s2,p)] = model.NewBoolVar("{};{}{};{}{}".format(tracer_names[pib],
                                                                                     sess_names[s1],
                                                                                     tracer_names[mk],
                                                                                     sess_names[s2],
                                                                                     scanner_names[p]))
pib_mk_same_day={}
#For same participant splits where a PIB is followed by an MK6240 in the same day
for p in scanners:
    for s1 in range(len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan
        for s2 in range(s1+pib_dur,len(sessions)): #MK6240 scan must start after end of PIB
            for d in days:
                pib_mk_same_day[(pib,s1,d,mk,s2,d,p)] = model.NewBoolVar("{}:{}{};;{}:{}{}".format(tracer_names[pib],
                                                                                                sess_names[s1],
                                                                                                day_names[d],
                                                                                                tracer_names[mk],
                                                                                                sess_names[s2],
                                                                                                day_names[d],
                                                                                                scanner_names[p]))         
            
###Could remove extra variables for MK sessions before 12:30 or 11:30 Mondays
###Add variables for MK Followed by PIB
mk_pib_concur={}             
for p in scanners:
    for s1 in range(8,len(sessions)): #avoid sessions that could not be followed by an MK6240 scan
        for s2 in range(len(sessions)-10): #MK6240 scan must start after end of PIB
            mk_pib_concur[(mk,s1,pib,s2,p)] = model.NewBoolVar("{};{}{};{}{}".format(tracer_names[mk],
                                                                                     sess_names[s1],
                                                                                     tracer_names[pib],
                                                                                     sess_names[s2],
                                                                                     scanner_names[p]))
mk_then_pib={}
#For splits with an afternoon MK6240 scan followed by a next day, morning PIB
for p in scanners:
    for s1 in range(8,len(sessions)): #MK6240 afternoon sessions only
        for s2 in range(len(sessions)-10): #PIB scans only starting at 12pm or sooner
            for d in days[0:len(days)-1]:
                mk_then_pib[(mk,s1,d,pib,s2,d+1,p)] = model.NewBoolVar("{}:{}{};;{}:{}{}".format(tracer_names[mk],
                                                                                                sess_names[s1],
                                                                                                day_names[d],
                                                                                                tracer_names[pib],
                                                                                                sess_names[s2],
                                                                                                day_names[d+1],
                                                                                                scanner_names[p]))

In [ ]:
#Allows us to sum instances when there are multiple MK6240 scans in a day
mk = tracer_names.index("MK6240")
mk_dur=scan_durations["MK6240"]
mk_sess_by_day = {}
for s in sessions:
    for d in days:
        for p in scanners:
            mk_sess_by_day[(mk,s,d,p)] = model.NewBoolVar("{}{}{}{}".format(tracer_names[mk],
                                                                            day_names[d],
                                                                            sess_names[s],
                                                                            scanner_names[p]))
mk_concur = {}
for s1 in range(7,len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan and cannot have MK6240
    for s2 in range(s1+mk_dur,len(sessions)): #MK6240 scan must start after end of first MK6240
        for p in scanners:
            mk_concur[(mk,s1,s2,p)] = model.NewBoolVar("{};{};{};{}".format(tracer_names[mk],
                                                                            sess_names[s1],
                                                                            sess_names[s2],
                                                                            scanner_names[p]))
mk_same_day = {}
for s1 in range(7,len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan
    for s2 in range(s1+mk_dur,len(sessions)): #MK6240 scan must start after end of first MK6240
        for d in days:
            for p in scanners:
                mk_same_day[(mk,s1,s2,d,p)] = model.NewBoolVar("{};{}{};{};{}".format(tracer_names[mk],
                                                                                       sess_names[s1],
                                                                                       sess_names[s2],
                                                                                       day_names[d],
                                                                                       scanner_names[p]))

In [ ]:
#Allows us to sum instances when there are multiple PIB scans in a day
pib = tracer_names.index("PIB")
pib_dur=scan_durations["PIB"]
pib_sess_by_day = {}
for s in sessions:
    for d in days:
        for p in scanners:
            pib_sess_by_day[(pib,s,d,p)] = model.NewBoolVar("{}{}{}{}".format(tracer_names[pib],
                                                                              day_names[d],
                                                                              sess_names[s],
                                                                              scanner_names[p]))
pib_concur = {}             
for s1 in range(len(sessions)-pib_dur): #avoid sessions that could not be followed by an PIB scan
    for s2 in range(s1+pib_dur,len(sessions)): #PIB scan must start after first PIB
        for p in scanners:
            pib_concur[(pib,s1,s2,p)] = model.NewBoolVar("{};{}{};{}".format(tracer_names[pib],
                                                                              sess_names[s1],
                                                                              sess_names[s2],
                                                                              scanner_names[p]))
pib_same_day = {}
for s1 in range(len(sessions)-pib_dur): #avoid sessions that could not be followed by an PIB scan
    for s2 in range(s1+pib_dur,len(sessions)): #PIB scan must start after first PIB
        for d in days:
            for p in scanners:
                pib_same_day[(pib,s1,s2,d,p)] = model.NewBoolVar("{};{}{};{};{}".format(tracer_names[pib],
                                                                                         sess_names[s1],
                                                                                         sess_names[s2],
                                                                                         day_names[d],
                                                                                         scanner_names[p]))

In [ ]:
### Hard Constraints ###

In [ ]:
#Constraints creating distinct sessions

In [ ]:
#Each session on a scanner gets at most 1 tracer
for p in scanners:
    for d in days:
        for s in sessions:
            model.Add(sum(x[(t,d,s,p)] for t in tracers) <= 1)

In [ ]:
#Prevent overlap between scans
for p in scanners:
    for d in days:
        for s in sessions[0:len(sessions)-1]:
            overlap = []
            for t in tracers:
                value=scan_durations[tracer_names[t]]
                for sess in range(value):
                    if s+sess < len(sessions)-(value-1): ###TODO: Find a better way to account for 2 or 3 value without going out of range
                        overlap.append(x[(t,d,s+sess,p)])
            model.Add(sum(overlap) <= 1)
                
#IDR why I used nested lists inside of overlap but we will try not doing that and see if it works                
#for p in scanners:
#    for d in days:
#        for s in sessions[0:14]:
#            overlap = []
#            for t in tracers:
#                overlap.extend([
#                    x[(t,d,s,p)],x[(t,d,s+1,p)],
#                    x[(t,d,s+2,p)],x[(t,d,s+3,p)]
#                ])
#            model.Add(sum(overlap) <= 1)

In [ ]:
#Prevent scans from being scheduled in a slot where they will finish after 17:30
too_late=[]
for p in scanners:
    for d in days:
        for t in tracers:
            value=scan_durations[tracer_names[t]]
            for s in sessions[len(sessions)-(value-1):len(sessions)]:
                too_late.append(x[(t,d,s,p)])
model.Add(sum(too_late) == 0)

In [ ]:
#Each tracer gets at least 1 session per week
#for t in tracers:
#    all_sess = []
#    for p in scanners:
#        for d in days:
#            for s in sessions:
#                all_sess.append(x[(t,d,s,p)])
#    model.Add(sum(all_sess) >= 1)

In [ ]:
#PIB constraints

In [ ]:
#PIB 3 max per day/batch
pib = tracer_names.index("PIB")
for d in days[1:4]:
    p_day = []
    for p in scanners:
        for s in sessions:
            p_day.append(x[(pib,d,s,p)])
    model.Add(sum(p_day) <= 3)

In [ ]:
#PIB 1 max per Monday
pib = tracer_names.index("PIB")
d = day_names.index("Mon")
p_mon = []
for p in scanners:
    for s in sessions:
        p_day.append(x[(pib,d,s,p)])
model.Add(sum(p_day) <= 1)

In [ ]:
#PIB 3 hr gap between PIB scans
three_hours = 6 #sessions
pib = tracer_names.index("PIB")
for d in days:
    for s in sessions[0:len(sessions)-three_hours]:
        p_gap = []
        for p in scanners:
            for num in range(three_hours):
                p_gap.append(x[(pib,d,s+num,p)])
        model.Add(sum(p_gap) <= 1)

In [ ]:
#PIB 1 hr gap between PIB scans 9:00-11:00; 10:00-12:00 - Use both scanners

###TODO####

In [ ]:
#AV1451 constraints

In [ ]:
#12 per month for AV1451; Average to 3 per week for weekly model
av = tracer_names.index("AV1451")
a_week = []
for p in scanners:
    for d in days:
        for s in sessions:
            a_week.append(x[(av,d,s,p)])
model.Add(sum(a_week) <= 3)

In [ ]:
#24HR between production but 2 scans per production batch/day 
av = tracer_names.index("AV1451")
for d in days:
    a_2 = []
    for p in scanners:
        for s in sessions:
            a_2.append(x[(av,d,s,p)])
    model.Add(sum(a_2) <= 2)

In [ ]:
#12:00 earliest sess for AV1451
av = tracer_names.index("AV1451")
a_early = []
for p in scanners:
    for d in days:
        for s in list(sessions[0:8]):
            a_early.append(x[(av,d,s,p)])
model.Add(sum(a_early) == 0)

In [ ]:
#MK6240 constraints

In [ ]:
#MK6240 max 3 sessions each day
mk = tracer_names.index("MK6240")
for d in days:
    m_3 = []
    for p in scanners:
        for s in sessions:
            m_3.append(x[(mk,d,s,p)])
    model.Add(sum(m_3) <= 3)

In [ ]:
#MK6240 only sessions 9 through 17; Tues-Fri
mk = tracer_names.index("MK6240")
m_days = []
#sess_not = [0,1,2,3,4,5,6,7,8,18]
for p in scanners:
    for d in days[1:5]:
        for s in list(sessions[0:9])+list(sessions[18:19]):
            m_days.append(x[(mk,d,s,p)])
model.Add(sum(m_days) == 0)

In [ ]:
#MK6240 only sessions 8 through 14; Mon
mk = tracer_names.index("MK6240")
d = day_names.index("Mon")
m_mon = []
for p in scanners:
    for s in list(sessions[0:7])+list(sessions[15:19]):
        m_mon.append(x[(mk,d,s,p)])
model.Add(sum(m_mon) == 0)

In [ ]:
#UCB-J constraints

In [ ]:
#UCB-J max 1 session each day
ucb_j = tracer_names.index("UCB-J")
for d in days:
    u_1 = []
    for p in scanners:
        for s in sessions:
            u_1.append(x[(ucb_j,d,s,p)])
    model.Add(sum(u_1) <= 1)

In [ ]:
#UCB-J session 6 or 12 Tues-Fri
ucb_j = tracer_names.index("UCB-J")
for d in days[1:5]:
    u_sess = []
    for p in scanners:
        for s in list(sessions[0:5])+list(sessions[6:11])+list(sessions[12:19]):
            u_sess.append(x[(ucb_j,d,s,p)])
    model.Add(sum(u_sess) == 0)

In [ ]:
#UCB-J session 14 Mon
ucb_j = tracer_names.index("UCB-J")
d = day_names.index("Mon")
u_mon = []
for p in scanners:
    for s in list(sessions[0:13])+list(sessions[14:19]):
        u_mon.append(x[(ucb_j,d,s,p)])
model.Add(sum(u_mon) == 0)

In [ ]:
#UCB-J 90-min gap prior to session during which other 11C tracers cannot be scanned [PIB,ER176,UCB-J]
#UCB-J constraint already included with 1 session per day constraint
#ER176 is not considered in this model
ucb_j = tracer_names.index("UCB-J")
pib = tracer_names.index("PIB")
for d in days[1:5]: #Tues-Fri
    u_sess = []
    for p in scanners:
        for s_ucb_j in [sessions[5],sessions[11]]: #only sessions 6 and 12 on these days
            u_sess.append(x[(ucb_j,d,s_ucb_j,p)])
            #3 sessions before these sessions must be not have scans for [PIB,ER176,UCB-J]
            for blocked in range(1,4):
                u_sess.append(x[(pib,d,s_ucb_j-blocked,p)])
                u_sess.append(x[(ucb_j,d,s_ucb_j-blocked,p)])
    model.Add(sum(u_sess) == 0)

In [ ]:
#UCB-J 90-min gap prior to session during which other 11C tracers cannot be scanned [PIB,ER176,UCB-J]
#UCB-J constraint already included with 1 session per day constraint
#ER176 is not considered in this model
ucb_j = tracer_names.index("UCB-J")
pib = tracer_names.index("PIB")
d = day_names.index("Mon")
s_ucb_j = sessions[13]
u_mon = []
for p in scanners:
    #3 sessions before these sessions must be not have scans for [PIB,ER176,UCB-J]
    u_mon.append(x[(ucb_j,d,s_ucb_j,p)])
    for blocked in range(1,4):
        u_mon.append(x[(pib,d,s_ucb_j-blocked,p)])
model.Add(sum(u_mon) == 0)

In [ ]:
#Constraints for decision variables that link outcomes/concurrent end states

In [ ]:
#Incentivize having PIB scan followed by MK6240 in same day
pib = tracer_names.index("PIB")
mk = tracer_names.index("MK6240")
pib_dur=scan_durations["PIB"]
mk_dur=scan_durations["MK6240"]
#Link pib_mk_concur with pib_mk_sess_by_day
for s1 in range(len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan
    for s2 in range(s1+pib_dur,len(sessions)): #MK6240 scan must start after end of PIB
        for p in scanners:
            for d in days:
                #Link pib_mk_same_day with pib_mk_sess_by_day. Keeps variables in sync
                pib_day = pib_mk_sess_by_day[(pib,s1,d,p)]
                mk_day = pib_mk_sess_by_day[(mk,s2,d,p)] #originally s1, as well. Mistake?
                same_day_pairing = pib_mk_same_day[(pib,s1,d,mk,s2,d,p)]
                model.AddBoolOr([pib_day.Not(), mk_day.Not(), same_day_pairing])
                # if same_day_pairing is True, then pib_day and mk_day must be True
                model.AddImplication(same_day_pairing, pib_day)
                model.AddImplication(same_day_pairing, mk_day)

            #Link pib_mk_concur with pib_mk_same_day
            pib_then_mk = sum(pib_mk_same_day[(pib,s1,d,mk,s2,d,p)] for d in days)
            model.Add(pib_then_mk == pib_mk_concur[(pib,s1,mk,s2,p)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having MK6240 scan followed by PIB the next morning
pib = tracer_names.index("PIB")
mk = tracer_names.index("MK6240")
#Link pib_mk_concur with pib_mk_sess_by_day
for s1 in range(8,len(sessions)): #MK6240 afternoon sessions only
    for s2 in range(len(sessions)-10): #PIB scans only starting at 12pm or sooner
        for p in scanners:
            for d in days[0:len(days)-1]:
                #Link pib_mk_same_day with pib_mk_sess_by_day. Keeps variables in sync
                pib_day = pib_mk_sess_by_day[(pib,s1,d,p)]
                mk_day = pib_mk_sess_by_day[(mk,s2,d,p)]
                same_day_pairing = mk_then_pib[(mk,s1,d,pib,s2,d+1,p)]
                model.AddBoolOr([pib_day.Not(), mk_day.Not(), same_day_pairing])
                # if same_day_pairing is True, then pib_day and mk_day must be True
                model.AddImplication(same_day_pairing, pib_day)
                model.AddImplication(same_day_pairing, mk_day)

            #Link pib_mk_concur with mk_then_pib
            mk_pib = sum(mk_then_pib[(mk,s1,d,pib,s2,d+1,p)] for d in days[0:len(days)-1])
            model.Add(mk_pib == mk_pib_concur[(mk,s1,pib,s2,p)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having multiple MK6240
mk = tracer_names.index("MK6240")
mk_dur=scan_durations["MK6240"]
#Link mk_concur with mk_sess_by_day
for s1 in range(7,len(sessions)-mk_dur): #avoid sessions that could not be followed by an MK6240 scan
    for s2 in range(s1+mk_dur,len(sessions)): #MK6240 scan must start after end of first MK6240
        for p in scanners:
            for d in days:
                #Link mk_same_day with mk_sess_by_day. Keeps variables in sync
                #for t in [t1,t2]:
                mk_1_day = mk_sess_by_day[(mk,s1,d,p)]
                mk_2_day = mk_sess_by_day[(mk,s2,d,p)]
                same_day_pairing = mk_same_day[(mk,s1,s2,d,p)]
                model.AddBoolOr([mk_1_day.Not(), mk_2_day.Not(), same_day_pairing])
                # if same_day_pairing is True, then mk_1_day and mk_2_day must be True
                model.AddImplication(same_day_pairing, mk_1_day)
                model.AddImplication(same_day_pairing, mk_2_day)

            #Link mk_concur with mk_same_day
            two_mk = sum(mk_same_day[((mk,s1,s2,d,p))] for d in days)
            model.Add(two_mk == mk_concur[(mk,s1,s2,p)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having multiple PIB
pib = tracer_names.index("PIB")
pib_dur=scan_durations["PIB"]
#Link av_concur with pib_sess_by_day
for s1 in range(len(sessions)-pib_dur): #avoid sessions that could not be followed by an PIB scan
    for s2 in range(s1+pib_dur,len(sessions)): #PIB scan must start after end of first PIB
        for p in scanners:
            for d in days:
                #Link pib_same_day with pib_sess_by_day. Keeps variables in sync
                #for t in [t1,t2]:
                pib_1_day = pib_sess_by_day[(pib,s1,d,p)]
                pib_2_day = pib_sess_by_day[(pib,s2,d,p)]
                same_day_pairing = pib_same_day[(pib,s1,s2,d,p)]
                model.AddBoolOr([pib_1_day.Not(), pib_2_day.Not(), same_day_pairing])
                # if same_day_pairing is True, then pib_1_day and pib_2_day must be True
                model.AddImplication(same_day_pairing, pib_1_day)
                model.AddImplication(same_day_pairing, pib_2_day)

            #Link pib_concur with pib_same_day
            two_pib = sum(pib_same_day[((pib,s1,s2,d,p))] for d in days)
            model.Add(two_pib == pib_concur[(pib,s1,s2,p)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
### Objective ###

In [ ]:
# pylint: disable=g-complex-comprehension
pib = tracer_names.index("PIB")
mk = tracer_names.index("MK6240")
pib_dur=scan_durations["PIB"]
mk_dur=scan_durations["MK6240"]
model.Maximize(
    sum(x[(t,d,s,p)] for t in tracers for d in days for s in sessions for p in scanners )
    + sum(pib_mk_weight * pib_mk_concur[pib,s,mk,s2,p] for s in range(len(sessions)-mk_dur) for s2 in range(s+pib_dur,len(sessions)) for p in scanners)
    + sum(mk_concur_weight * mk_concur[mk,s,s2,p] for s in range(7,len(sessions)-mk_dur) for s2 in range(s+mk_dur,len(sessions)) for p in scanners)
    + sum(mk_pib_weight * mk_pib_concur[(mk,s1,pib,s2,p)] for s1 in range(8,len(sessions)) for s2 in range(len(sessions)-10) for p in scanners)
    + sum(pib_concur_weight * pib_concur[pib,s,s2,p] for s in range(len(sessions)-pib_dur) for s2 in range(s+pib_dur,len(sessions)) for p in scanners)
)
#First sum incentivizes scheduling any scans
#Second sum incentivizes PIB followed by an MK
#Third sum incentivizes MK followed by a PIB
#Fourth sum incentivizes multiple MK6240 sessions in one day
#Fifth sum incentivizes multiple PIB sessions in one day
#Coefficients are arbitrary weights for incentivizing these sums relative to each other

In [ ]:
### Instanciate Solver, solve, and output end states ###

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)


if status == cp_model.OPTIMAL or status == 2:
    print('Solution:')
    results = []
    for d in days:
        print(day_names[d])
        day = ["Day{}".format(d)]
        for p in scanners:
            for t in tracers:
                for s in sessions:
                    if solver.Value(x[(t,d,s,p)]) == 1:
                        day.append([d,t,s,p,1])
                        if design_matrix[p][t][d][s] == 1:
                            #weighted
                            print(tracer_names[t], 'scheduled', sess_names[s], 'on', scanner_names[p])
                        else:
                            print(tracer_names[t], 'scheduled', sess_names[s], 'on', scanner_names[p],
                                  '(not requested).')
        print()
        results.append(day)
    pd.DataFrame(results).to_csv("schedule_maximize_solution.csv",index=False)
    #Prints CSV file of solution
elif status == 0:
    print('No optimal solution found before search limit reached!')
elif status == 1:
    print("Invalid model! Call 'ValidateCpModel(model_proto)' for detailed information.")
elif status == 3:
    print("Model proven infeasible.")

In [ ]:
### Create Gantt Graph of Solution ###

In [ ]:
sch_data = pd.DataFrame(columns=["Session","Start","Finish","Tracer","color"])
#seed data with a background color for each session if no scan
for entry in range(len(sess_names)):
    sch_data.loc[len(sch_data.index)] = dict(Session=sess_names[entry], Start=0, Finish=5, Tracer="", color="1")
    
#Graph solution
#Read solution from CSV and then add scheduled sessions to Gantt
sch = pd.read_csv("schedule_maximize_solution.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
tracers_colors = ["2","3","4","5"]
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[4]) == 1 and int(ent_l[3]) == 0: #Get first scanner schedule
                dur = scan_durations[tracer_names[int(ent_l[1])]] #get how many sessions this scan will last
                for s in range(dur):
                    sch_entry = dict(Session=sess_names[int(ent_l[2])+s], #add entry for each 30min of scan
                                    Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                    Tracer=tracer_names[int(ent_l[1])], 
                                     color=tracers_colors[int(ent_l[1])])
                    sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig_hr = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig_hr.update_yaxes(autorange="reversed")
fig_hr.layout.xaxis.type = 'linear'
for d in fig_hr.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()
fig_hr.show()

In [ ]:
#Save Gantt Graph as HTML
hr_name = 'scheduling_max_graph_hr'
fig_hr.write_html(f"{hr_name}.html")

In [ ]:
sch_data = pd.DataFrame(columns=["Session","Start","Finish","Tracer","color"])
#seed data with a background color for each session if no scan
for entry in range(len(sess_names)):
    sch_data.loc[len(sch_data.index)] = dict(Session=sess_names[entry], Start=0, Finish=5, Tracer="", color="1")

#Graph solution
#Read solution from CSV and then add scheduled sessions to Gantt
sch = pd.read_csv("schedule_maximize_solution.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
tracers_colors = ["2","3","4","5"]
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[4]) == 1 and int(ent_l[3]) == 1: #Get second scanner schedule
                dur = scan_durations[tracer_names[int(ent_l[1])]] #get how many sessions this scan will last
                for s in range(dur):
                    sch_entry = dict(Session=sess_names[int(ent_l[2])+s],
                                    Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                    Tracer=tracer_names[int(ent_l[1])], 
                                     color=tracers_colors[int(ent_l[1])])
                    sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig_bio = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig_bio.update_yaxes(autorange="reversed")
fig_bio.layout.xaxis.type = 'linear'
for d in fig_bio.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()
fig_bio.show()

In [ ]:
#fig.to_dict() #use if want to adjust formatting

In [ ]:
#Save Gantt Graph as HTML
bio_name = 'scheduling_max_graph_bio'
fig_bio.write_html(f"{bio_name}.html")